# Chain-of-Zoom

Revisión informal de la arquitectura "Chain-of-Zoom" (_CoZ_).

_Framework_ publicado por KAIST AI en mayo de 2025.

- [https://arxiv.org/pdf/2505.18600](https://arxiv.org/pdf/2505.18600)

Describe un proceso iterativo para aumentar la resolución de imágenes.

![Chain-of-Zoom](img/chain-of-zoom-01.png)

## 1. Arquitectura

Los modelos existentes aumentan la resolución de las imágenes en un factor de escala fijo con el que fueron entrenados, produciendo imágenes de peor calidad para otros factores de escala.

```
Low Resolution ─> Super-Resolution Model ─> High Resolution
```

_CoZ_ propone utilizar un _pipeline_ de N pasos intermedios que aumentan la resolución progresivamente ayudados por un modelo de visión-lenguaje que describe el contenido de la imagen en cada paso.

```
                       ┌─────────────────────────────────────────┐
                       ↓                                         ↑
Low Resolution ─> N x (─> Super-Resolution Model ─> Scale-State) ─> High Resolution
                       ↓             ↑
           Vision-Language Model ─> Prompt
```

## 2. Análisis

Se demuestra que la arquitectura es independiente de los modelos utilizados y no requiere reentrenarlos, pero se recomienda hacer _fine-tuning_ del modelo de visión-lenguaje para obtener mejores resultados.

### 2.1. Scale-States

_CoZ_ plantea el problema de generar una imagen de alta resolución $x_H$ a partir de otra de baja resolución $x_L$ utilizando un modelo probabilístico. Es decir, la probabilidad condicionada de generar $x_H$ si se dispone de $x_L$. O dicho de otra forma, la probabilidad de generar una imagen en alta resolución a partir de una dada en baja resolución.

- $ p(x_H \mid x_L) $

Y lo hace proponiendo generar $n$ imágenes intermedias $x_i$, en una secuencia $(x_0, x_1, \ldots, x_n)$, aumentando la resolución en cada paso por un factor de escala $s$.

- $ x_0 = x_L $

- $ x_n = x_H $

Dimensiones:

- $ x_L \in \mathbb{R}^{d_0} $

- $ x_i \in \mathbb{R}^{d_i} $

- $ x_H \in \mathbb{R}^{d_n} $

Siendo:

- $ d_i = s d_{i-1} $

A estos pasos intermedios los denomina _scale-states_ (estados de escala intermedios).

### 2.2. AR-2

El marco teórico del _paper_ parte de la distribución _conjunta_ en el contexto de cadenas de Markov. Es decir, de la secuencia de estados de un sistema donde la probabilidad del estado $x_i$ depende solo del estado anterior $x_{i-1}$. O dicho de otra forma, la probabilidad de generar una imagen con mayor resolución que la anterior.

- $ p(x_0, x_1, \ldots, x_n) = p(x_0) \prod\limits^n_{i=1} p(x_i \mid x_{i−1}) $

El problema de esta aproximación es que no resulta óptima para la ampliación de imágenes, porque se van perdiendo detalles de alta frecuencia a medida que se avanza por la secuencia, por lo que se plantea una solución que tiene en cuenta los dos estados anteriores $x_{i-1}$ y $x_{i-2}$. Es decir, las dos imágenes anteriores.

- $ p(x_0, x_1, \ldots, x_n) = p(x_0, x_1) \prod\limits^n_{i=2} p(x_i \mid x_{i−1}, x_{i−2}) $

A este modelo _autorregresivo_ (_AR_, por sus siglas en inglés), que toma en cuenta los dos estados anteriores, se denomina _AR-2_.

### 2.3. Variables Latentes

El modelo _AR-2_ propuesto sigue sin ser un sistema eficiente, porque a medida que se escalan las imágenes se pierde el contexto de la imagen original a partir de la cual se generaron, por lo que se plantea añadir más información en cada paso. Técnicamente, introduciendo variables latentes $c_i$ que ayuden en el cálculo. Y más concretamente, añadiendo las descripciones de las imágenes (_prompts_) generadas por un modelo de visión-lenguaje.

- $ p(x_i \mid x_{i−1}, x_{i−2}) = \displaystyle \int p(x_i \mid x_{i−1}, x_{i−2}, c_i) p(c_i \mid x_{i-1}, x_{i-2}) dc_i $
 
Esta expresión representa la distribución _marginal_ de $x_i$ condicionada a $x_{i-1}$ y $x_{i-2}$. Es decir, la probabilidad teniendo en cuenta los dos estados anteriores y "marginando" las variables latentes. O dicho de otra forma, la probabilidad de generar una imagen en base a las dos anteriores promediando el efecto de las descripciones según su propia probabilidad. No es que las descripciones se ignoren, se utilizan en el cálculo, pero como estados ocultos, lo que interesa es la imagen a generar.

La integral de la expresión es porque $c_i$  se considera una variable aleatoria continua, existe en todo el intervalo. En un apéndice del _paper_ se proporciona la demostración de porqué se puede utilizar esta expresión en la distribución conjunta eliminando la integral.

### 2.4. Función Objetivo

A raíz del planteamiento anterior, el _paper_ continúa con la distribución conjunta de una secuencia de estados $x_i$ que siguen la estructura _AR-2_ y variables latentes $c_i$. Es decir, la probabilidad de generar a cada paso de la secuencia una imagen de mayor resolución en función de las dos imágenes anteriores y su descripción.

- $ p(x_0, c_1, x_1, \ldots, c_n, x_n) = p(x_0, x_1) \prod\limits^n_{i=2} p(x_i \mid x_{i−1}, x_{i−2}, c_i) p(c_i \mid x_{i-1}, x_{i-2}) $

En el primer paso $i = 1$ el estado $x_{i-2}$ no está disponible, por lo que el _paper_ simplifica el cálculo de los primeros términos de la secuencia tomando en cuenta sólo la imagen inicial.

- $ p(c1 \mid x0) $

- $ p(x1 \mid x0, c1) $

Y como es práctica habitual, se aplica el logaritmo para convertir la productoria en sumatorias, ya que el logaritmo de un producto es la suma de los logaritmos de los factores.

- $ \log(ab) = \log(a) + \log(b) $

Lo que define la función objetivo que busca maximizar la _log-verosimilitud_ total. Es decir, entrenar el modelo para que aprenda cuales combinaciones de $x_i$ e $c_i$ tienen más sentido, porque su probabilidad conjunta, condicionada por los pasos anteriores, es alta.

- $ \mathcal{L} = \log p(x_0) + \underbrace{\sum\limits^n_{i=1} \log p(x_i \mid x_{i−1}, x_{i−2}, c_i)}_{\mathcal{L}_{SR}} + \underbrace{\sum\limits^n_{i=1} \log p(c_i \mid x_{i−1}, x_{i−2})}_{\mathcal{L}_{VLM}} $

El primer sumatorio representa el objetivo a maximizar como parte del entrenamiento del modelo de _Super-Resolution_ (_SR_).

El segundo sumatorio representa el objetivo a maximizar como parte del entrenamiento del modelo de _Vision-Language_(_VLM_).

## 3. Super-Resolution Model

### 3.1. Distribución Gausiana

$\mathcal{L}_{SR}$ representa la log-verosimilitud de generar la imagen $x_i$ dadas las dos imágenes previas $x_{i-1}$ y $x_{i-2}$ y la descripción $c_i$ de $x_i$.

- $ p(x_i \mid x_{i−1}, x_{i−2}, c_i) $

El _paper_ asume en este punto que la distribución de probabilidad es gausiana. Es decir, una distribución normal con una determinada media y varianza.

- $ p(x_i \mid x_{i-1}, x_{i-2}, c_i) := \mathcal{N} (x_i ; f_{\theta}(x_{i-1}, x_{i-2}, c_i), \sigma^2 \mathbf{I}) $

Siendo:

- $ f_{\theta}(x_{i-1}, x_{i-2}, c_i) $ la media.

- $ \sigma^2 \mathbf{I} $ la varianza.

Donde $f_{\theta}$ es la función que implementa el modelo $\theta$ de super-resolución. Es decir, el que genera la imagen $x_i$. Siendo la imagen un vector de $d$ dimensiones que representa la media para cada variable (píxel).

Y donde $\mathbf{I}$ es la matriz identidad. Es decir, que al tratarse de una distribución gaussiana multivariada, la varianza no es un único número escalar, sino una matriz de covarianza de $d \times d$ dimensiones. Con valores únicamente sobre la diagonal principal, por lo que las variables (píxeles) se consideran independientes.

### 3.2. Densidad Gausiana

La distribución puede expresarse a través de la correspondiente función de densidad de la distribución gausiana multivariada (_PDF_). Es decir, con una expresión que permite calcular la probabilidad de que ocurra un determinado valor $x$ dada una media $\mu$ y una matriz de covarianza $\Sigma$ determinadas.

- $ \mathcal{N}(x; \mu, \Sigma) = \cfrac{1}{(2 \pi)^{d/2} \mid \Sigma \mid^{1/2}} \exp{\left(-\cfrac{1}{2}(x - \mu)^T \Sigma^{-1} (x - \mu)\right)} $

Donde la forma $\sigma^2 \mathbf{I}$ de la varianza permite simplificar algunos términos.

- $ \mid \Sigma \mid = \mid \sigma^2 \mathbf{I} \mid = (\sigma^2)^d \mid \mathbf{I} \mid = \sigma^{2d} $

- $ \Sigma^{-1} = \cfrac{1}{\sigma^2} \mathbf{I} $

Sustituyendo en la expresión.

- $ \mathcal{N}(x; \mu, \Sigma) = \cfrac{1}{(2 \pi \sigma^2)^{d/2}} \exp{\left(-\cfrac{1}{2 \sigma^2}(x - \mu)^T (x - \mu)\right)} $

### 3.3. Norma L2

El último término entre paréntesis de la función de densidad es igual a calcular la suma de las diferencias al cuadrado entre los vectores $x$ y $\mu$. Es decir, calcular la norma euclidiana (o norma _L2_) al cuadrado.

- $ (x - \mu)^T (x - \mu) = (x - \mu)^T (x - \mu) = (x_1 - \mu_1)(x_1 - \mu_1) + (x_2 - \mu_2)(x_2 - \mu_2) + \ldots + (x_d - \mu_d)(x_d - \mu_d) = (x_1 - \mu_1)^2 + (x_2 - \mu_2)^2 + \ldots + (x_d - \mu_d)^2 = \lVert x - \mu \rVert^2 $

Sustituyendo en la expresión.

- $ \mathcal{N}(x; \mu, \Sigma) = \cfrac{1}{(2 \pi \sigma^2)^{d/2}} \exp{\left(-\cfrac{1}{2 \sigma^2} \lVert x - \mu \rVert^2 \right)} $

### 3.4. Log-Verosimilitud

Finalmente, la expresión de la función de densidad permite calcular la probabilidad buscada.

- $ p(x_i \mid x_{i-1}, x_{i-2}, c_i) = \cfrac{1}{(2 \pi \sigma^2)^{d/2}} \exp{\left(-\cfrac{1}{2 \sigma^2} \lVert x_i - f_{\theta}(x_{i-1}, x_{i-2}, c_i) \lVert^2 \right)} $

Y aplicando logaritmo, que elimina la exponencial, se obtiene la _log-verosimilitud_ de la función objetivo para el modelo de super-resolución.

- $ \log p(x_i \mid x_{i-1}, x_{i-2}, c_i) = \log \cfrac{1}{(2 \pi \sigma^2)^{d/2}} \exp \left(-\cfrac{1}{2 \sigma^2} \lVert x_i - f_{\theta}(x_{i-1}, x_{i-2}, c_i) \lVert^2 \right) = - \cfrac{1}{2 \sigma^2} \lVert x_i - f_{\theta}(x_{i-1}, x_{i-2}, c_i) \rVert^2 - \cfrac{d}{2} \log(2 \pi \sigma^2) $

Siendo el último sumando un valor constante $C = - \cfrac{d}{2} \log(2 \pi \sigma^2)$.

- $ \log p(x_i \mid x_{i-1}, x_{i-2}, c_i) = - \cfrac{1}{2 \sigma^2} \lVert x_i - f_{\theta}(x_{i-1}, x_{i-2}, c_i) \rVert^2 + C $

### 3.5. Modelo Agnóstico

Los modelos de super-resolución se alimentan con una imagen y un texto, y generan una imagen de mayor resolución de salida. Por lo que se plantea el dilema de tener que entrenar modelos con dos imágenes de entrada $x_{i-1}$ y $x_{i-2}$. Sin embargo, la solución propuesta en el _paper_ es que el modelo de super-resolución acceda a la imagen $x_{i-2}$ solamente a través de la descripción $c_i$.

- $ c_i = c_i (x_{i-1}, x_{i-2}) $

- $ f_{\theta}(x_{i-1}, x_{i-2}, c_i) = f_{\theta}(x_{i-1}, c_i (x_{i-1}, x_{i-2})) $

- $ \log p(x_i \mid x_{i-1}, x_{i-2}, c_i) = - \cfrac{1}{2 \sigma^2} \lVert x_i - f_{\theta}(x_{i-1}, c_i (x_{i-1}, x_{i-2})) \rVert^2 + C $

Por lo que el _paper_ concluye afirmando que maximizar la función objetivo $\mathcal{L_{SR}}$ es lo mismo que minimizar el error cuadrático medio (_MSE_) entre $x_i$ y la predicción $f_{\theta}$ del modelo. Es decir, la función tiene un signo negativo, y la constante $C$ es positiva, por lo que cuanto menor sea el error cuadrático medio, representado por la suma de los errores cuadráticos $\lVert x_i - f_{\theta}(x_{i-1}, c_i (x_{i-1}, x_{i-2})) \rVert^2$, mayor será el valor de la función.

Y minimizar el error cuadrático medio es precisamente para lo que la mayoría de modelos de super-resolución están entrenados. Por lo que se puede utilizar cualquier modelo ya pre-entrenado sin necesidad de volver a entrenarlos de una manera específica.

_CoZ_ es agnóstico respecto al modelo de super-resolución utilizado.

## 4. Vision-Language Model

### 4.1. Log-Verosimilitud

$\mathcal{L}_{VLM}$ representa la log-verosimilitud de generar la descripción $c_i$ dadas las dos imágenes previas $x_{i-1}$ y $x_{i-2}$.

- $ p(c_i \mid x_{i−1}, x_{i−2}) $

Siendo una descripción $c_i$ una secuencia de $T_i$ _tokens_, o sus correspondientes _embeddings_.

- $ c_i = (c_{i,1}, c_{i,2}, \ldots, c_{i,T_i}) $

Lo que sabemos que puede expresarse mediante una distribución de probabilidad conjunta de manera autorregresiva.

- $ p(c_i \mid x_{i−1}, x_{i−2}) = \prod\limits^{T_i}_{t=1} p(c_{i,t} \mid x_{i−1}, x_{i−2}, c_{i,<t}) $

Siendo $ c_{i,<t} = (c_{i,1}, c_{i,2}, \ldots, c_{i,t-1}) $ los _tokens_ anteriores de la secuencia.

### 4.2. Modelo Agnóstico

Para entrenar un modelo de visión-lenguaje $\phi$ que genere secuencias de _tokens_ de manera autorregresiva, el objetivo estándar es maximizar la log-verosimilitud de las secuencias generadas.

- $ \log p_{\phi}(c_i \mid x_{i-1}, x_{i-2}) = \sum\limits_{t=1}^{T_i} \log p_{\phi}(c_{i,t} \mid x_{i-1}, x_{i-2}, c_{i,<t}) $

Lo que equivale a minimizar la log-verosimilitud negativa de las secuencias.

- $ -\log p_{\phi}(c_i \mid x_{i-1}, x_{i-2}) = - \sum\limits_{t=1}^{T_i} \log p_{\phi}(c_{i,t} \mid x_{i-1}, x_{i-2}, c_{i,<t}) $

Que es precisamente para lo que están entrenados la mayoría de los modelos de visión-lenguaje. Habitualmente utilizando la función de pérdida de entropía cruzada. Por lo que se puede utilizar cualquier modelo ya pre-entrenado (que admita dos imágenes de entrada) con este objetivo, sin necesidad de reentrenarlo específicamente.

_CoZ_ es agnóstico respecto al modelo de visión-lenguaje utilizado.

## 5. Inferencia

### 5.1. Chain-of-Zoom

El proceso de inferencia comienza con la imagen de entrada en baja resolución.

- $ x_0 = x_L $

La imagen se introduce en el modelo de visión-lenguaje para obtener una descripción.

- $ c_1 \sim p_\phi(c_1 \mid x_0) $

Y la imagen y la descripción se introducen en el modelo de super-resolución para obtener la primera imagen con resolución aumentada.

- $ x_1 \sim p_\theta(x_1 \mid x_0, c_1) $

El resto de imágenes se obtienen repitiendo el proceso de manera iterativa, pero alimentando el modelo de visión-lenguaje con las dos últimas imágenes en vez de sólo la anterior.

- $ c_i \sim p_\phi(c_i \mid x_{i-1}, x_{i-2}) $

- $ x_i \sim p_\theta(x_i \mid x_{i-1}, c_i) $

_Prompt_ utilizado para el modelo de visión-lenguaje:

```
The second image is a zoom-in of the first image.
Based on this knowledge, what is in the second image?
Give me a set of words.
```

## 6. Entrenamiento

A medida que aumenta la resolución de las imágenes el proceso se apoya cada vez más en las descripciones extraídas de las imágenes (_prompts_), por lo que es importante que los _prompts_ sean de buena calidad y se alineen con las expectativas para reducir el riesgo de generar _alucinaciones_.

### 6.1. Reinforcement Learning (RL)

El _paper_ propone realizar _fine-tuning_ del modelo de visión-lenguaje utilizando _Reinforcement Learning_ (_RL_).

```
Images ───────────────┬────────────────────────────────┐
                      ↓                                ↓
           Vision-Language Model ─> Prompt ─> Critic Vision-Language Model ─> Score
```

En cada paso se alimenta el modelo de visión-lenguaje con las dos imágenes anteriores generadas.

Y el _prompt_ generado se pasa a un segundo modelo de visión-lenguaje, entrenado con un número mayor de parámetros, que actúa como un "crítico" que le asigna una puntuación.

_Prompt_ utilizado para el modelo de visión-lenguaje crítico:

```
First Image: <image>
Second Image: <image>
The second image is a zoom-in of the first image. Please rate the quality of the following description
on how well it describes the second image. Output only a single score between 0 and 100.
Description: <Output of Base VLM>
Rating (0-100):
```

### 6.2. Generalized Reward Policy Optimization (GRPO)

El _paper_ propone utilizar tres tareas con recompensas distintas utilizando _Generalized Reward Policy Optimization_ (_GRPO_).

- $ R(c_i) = w_{critic} R_{critic} + w_{phrase} R_{phrase} + w_{rep} R_{rep} $

Siendo:

- $c_i$ el _prompt_ generado en el paso _i_ por el modelo de visión-lenguaje.

- $R_{critic}$ (_Critic Preference Reward_) la recompensa asignada por el modelo de visión-lenguaje crítico  al _prompt_ generado.

- $w_{critic}$ el peso específico asignado a la recompensa $R_{critic}$.

- $R_{phrase}$ (_Phrase-Exclusion Reward_) la recompensa asignada por una tarea que busca determinadas palabras o expresiones en el _prompt_, como "_first image_" o "_second image_", generadas erróneamente cuando se ha pedido de manera explícita generar únicamente una lista de palabras.

- $w_{phrase}$ el peso asignado a la recompensa $R_{phrase}$.

- $R_{rep}$ (_Repetition Penalty_) la recompensa asignada por una tarea que penaliza el texto repetitivo, frases o palabras repetidas una y otra vez en el _prompt_, algo que puede suceder a medida que aumenta la resolución de las imágenes, lo que dificulta extraer una descripción de las mismas.

- $w_{rep}$ el peso asignado a la recompensa $R_{rep}$.

- $R(c_i)$ la recompensa total ponderada que guía el proceso de aprendizaje.

Los valores de las recompensas se normalizan a un rango de valores entre $0$ y $1$.

Y los pesos se inicializan manualmente a valores escalares concretos antes del entrenamiento.

## 7. Setup

El _paper_ incluye detalles de las arquitecturas concretas utilizadas para la realización de los experimentos de prueba.

### 7.1. Componentes

Modelo de Super-Resolución (_SR_):

- OSEDiff

Modelo de Difusión:

- Stable Diffusion 3.0 (modelo generativo para entrenar el modelo _SR_)

Modelos de Visión-Lenguaje (_VLM_):

- Qwen2.5-VL-3B-Instruct (_VLM_ para la extracción de _prompts_)

- InternVL2.5-8B (_VLM_ crítico para entrenar el extractor de _prompts_)

Extractor de _prompts_ alternativo (_Naive_):

- DAPE (_Degradation-Aware Prompt Extractor_)

### 7.2. Datasets

- LSDIR (para entrenamiento del modelo _SR_ y entrenamiento _GRPO_)

- FFHQ (un subconjunto de 10K imágenes únicamente, para entrenamiento del modelo _SR_ y entrenamiento _GRPO_)

- DIV2K (para evaluación)

- DIV8K (para evaluación)

### 7.3. Técnicas

Método de Optimización (para ajuste fino del _VLM_):

- GRPO (_Generalized Reward Policy Optimization_)

  - $ w_{critic} = 1,0 $

  - $ w_{phrase} = 0,5 $

  - $ w_{rep} = 0,5 $